In [1]:
import numpy as np
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_1D import *
from plu import *
from ids import *
%matplotlib inline

In [2]:
n = 4 #nmber of sampling points   
deg = 3 # Degree of polynomiapaired plu Last Checkpoint: 22 minutes ago (unsaved changes)l          
interval = [-3, 3]  # interval of x
x = np.linspace(interval[0], interval[1], n, dtype=np.float64)
p_size = deg+1

Idea is that we take slices of input tall matrix A in the form $A_{2j:, 2j:}$ step by step. At each step we are looking between all pairs for (2x2) matrix with maximal determinant. Then we perform permutations of raws and columns to get block raws together and perform elimination.

In [12]:
def pluq_ids(A, debug = True):
    def mov_permute(C, j, ind, m = 'P'):
        if m == 'P':
            temp = np.copy(C[ind,:])
            C[ind,:] = C[j, :]
            C[j, :] = temp
        if m == 'Q':
            temp = np.copy(C[:,ind])
            C[:,ind] = C[:,j]
            C[:,j] = temp
        return()  
    def mov_LU(C, j, ind_r, ind_c, m = 'U'):
        if m == 'U':
            temp = np.copy(C[ind_r,:])
            C[ind_r,:] = C[j, :]
            C[j, :] = temp
        
            temp = np.copy(C[:,ind_c])
            C[:, ind_c] = C[:,j]
            C[:, j] = temp
        if m=='L':
            temp = np.copy(C[ind_r,:j])
            C[ind_r,:j] = C[j, :j]
            C[j, :j] = temp

            temp = np.copy(C[:j,ind_c])
            C[:j, ind_c] = C[:j,j]
            C[:j, j] = temp  
            
    def ellimination(L,U,ind):
        k = L.shape[0]
        for i in range(ind+1, k):
            L[i,ind] = U[i,ind]/U[ind,ind]
            U[i,ind:] -= L[i,ind]*U[ind,ind:] 
        return ()    
    
    n, m = A.shape[0], A.shape[1]
    P = np.eye((n), dtype=float)
    L = np.eye((m), dtype=float)
    L_add = np.zeros((n-m, m), dtype=float)
    L = np.concatenate((L, L_add), axis = 0)
    U = np.copy(A)
    Q = np.eye((m), dtype=float)
    yx = np.array([0, 0], dtype=int)
    max_det = np.zeros((2), dtype=float)
    
    
    for j in range(0, m, 2):
        ### in each slice we are looking for 2x2 matrix with maxvol and memorize 'k' as a number of first row 
        ### 'k'- is a counter of pairs, i.e. with each slice there is 1 pair less
        max_det = np.zeros((2), dtype=float)
        for k in range(j,n,2):
            pair = np.concatenate((U[k,j:],U[k+1,j:])).reshape(2,m-j).T
            piv,_ = maxvol(pair)
            print pair
            if np.abs(np.linalg.det(pair[piv])) > max_det[0]:
                max_det[0], max_det[1] = np.abs(np.linalg.det(pair[piv])), k
         
        pair = np.concatenate((U[max_det[1].astype(int),j:],U[max_det[1].astype(int)+1,j:])).reshape(2,m-j).T
        piv,_ = maxvol(pair)
        piv.sort()  
        
        yx[0] = max_det[1] 
        yx[1] = piv[0]+ j
    
        if (debug):
            if np.linalg.det(pair[piv]) == max_det[0]:
                print('correct 2x2 matrix')
              
            print ('on the', j, 'slice')
            print ('best row block is', max_det[1].astype(int), max_det[1].astype(int) + 1)
            print ('column coordinates:', piv[0] + j, piv[1] + j)
            print ('maxvol 2x2 submatrix', pair[piv.T])
            print ('with det = ', max_det[0])
            print ('pivoting and permutations start...')
            
        ### U moving ###
        mov_LU(U,j,yx[0],yx[1])
        ####
        
        ### L moving ###
        mov_LU(L,j,yx[0],yx[1],m='L')
        ###
        
        ### P&Q moving ###
        mov_permute(P,j,yx[0])
        mov_permute(Q,j,yx[1], m='Q')
        
        ### make them all zeros! Below (j,j) element
        ellimination(L,U,j)     
        
        #choosing second element to pivot. 
        yx[0] = max_det[1] + 1 
        yx[1] = piv[1] + j
    

        ### U moving ###
        mov_LU(U,j+1,yx[0],yx[1])
        ####
        
        ### L moving ###
        mov_LU(L,j+1,yx[0],yx[1],m='L')
        ###
        
        ### P&Q moving ###
        mov_permute(P,j+1,yx[0])
        mov_permute(Q,j+1,yx[1], m='Q')
        
        ### make them all zeros! (Below (j+1,j+1) element) ###
        ellimination(L,U,j+1)
           
    return(P,L,U,Q)  

In [3]:
b = Gen_1D_coupled(p_size, x, debug=True)
b

number of points(l) = 4,  number of monoms(n_size) = 4


array([[   1.,   -3.,   17.,  -99.],
       [   0.,    1.,  -12.,  105.],
       [   1.,   -1.,    1.,   -1.],
       [   0.,    1.,   -4.,    9.],
       [   1.,    1.,    1.,    1.],
       [   0.,    1.,    4.,    9.],
       [   1.,    3.,   17.,   99.],
       [   0.,    1.,   12.,  105.]])

In [4]:
p,l,u,q = pluq_ids(b)

[[   1.    0.]
 [  -3.    1.]
 [  17.  -12.]
 [ -99.  105.]]
[[ 1.  0.]
 [-1.  1.]
 [ 1. -4.]
 [-1.  9.]]
[[ 1.  0.]
 [ 1.  1.]
 [ 1.  4.]
 [ 1.  9.]]
[[   1.    0.]
 [   3.    1.]
 [  17.   12.]
 [  99.  105.]]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 0, 1)
('column coordinates:', 2, 3)
('maxvol 2x2 submatrix', array([[  17.,  -12.],
       [ -99.,  105.]]))
('with det = ', 596.99999999999955)
pivoting and permutations start...
[[  17.   -3.    1.  -99.]
 [ -12.    1.    0.  105.]
 [   1.   -1.    1.   -1.]
 [  -4.    1.    0.    9.]
 [   1.    1.    1.    1.]
 [   4.    1.    0.    9.]
 [  17.    3.    1.   99.]
 [  12.    1.    0.  105.]]
[[  17.           -3.            1.          -99.        ]
 [   0.           -1.11764706    0.70588235   35.11764706]
 [   0.           -0.82352941    0.94117647    4.82352941]
 [   0.            0.29411765    0.23529412  -14.29411765]
 [   0.            1.17647059    0.94117647    6.82352941]
 [   0.            1.70588235   

array([[  5.77000000e+02,  -3.36300000e+03,   1.70000000e+01,
         -9.90000000e+01,   1.00000000e+00,  -3.00000000e+00],
       [  0.00000000e+00,   1.18900693e+03,   1.20415945e+01,
         -3.50069324e+01,   1.41421144e+00,  -3.24263432e+00],
       [  0.00000000e+00,   0.00000000e+00,  -6.81171506e+01,
          3.96027969e+02,  -7.99994170e+00,   2.43430036e+01],
       [  0.00000000e+00,   1.81898940e-12,   0.00000000e+00,
         -1.40017841e+02,   0.00000000e+00,  -8.60660125e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.11022302e-16,
          0.00000000e+00,   1.10074873e+00,  -3.18932535e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.16941929e+00]])

In [101]:
def puq_s(A):
    n = A.shape[0]
    P = np.eye((n), dtype=float)
    L = np.eye((n), dtype=float)
    U = np.copy(A)
    Q = np.eye((n), dtype=float)
    yx = np.array([0, 0], dtype=int)
    for j in range(0, n):
        loc_max = np.argmax(np.abs(U[j:, j:]))
        yx[0] = loc_max / (n - j)
        yx[1] = loc_max % (n-j)
        print yx
        
       
        ### U moving ###
        temp = np.copy(U[j+yx[0],:])
        U[j+yx[0],:] = U[j, :]
        U[j, :] = temp
               
        temp = np.copy(U[:, j+yx[1]])
        U[:,j+yx[1]] = U[:, j]
        U[:, j] = temp 
        ####

        print ('uu',U)
        
        ### P&Q moving ###
        temp = np.copy(P[j+yx[0],:])
        P[j+yx[0],:] = (P[j,:])
        P[j,:] = temp 
          
        temp = np.copy(Q[:, j+yx[1]])
        Q[:,j+yx[1]] = (Q[:, j])
        Q[:, j] = temp  

        ###
        print (P)
    return(P, U, Q)  

In [14]:
rara = np.random.randint(1,18, size=(8, 4))
rara

array([[ 3, 12,  2,  2],
       [ 6, 15, 11, 15],
       [11, 16,  5, 13],
       [ 2, 15,  9,  4],
       [ 6, 14,  4, 15],
       [ 5, 12,  6, 11],
       [17,  7,  8, 15],
       [ 1,  4, 15, 15]])

In [15]:
p,l,u,q = pluq_ids(rara.astype(float),debug=True)

[[  3.   6.]
 [ 12.  15.]
 [  2.  11.]
 [  2.  15.]]
[[ 11.   2.]
 [ 16.  15.]
 [  5.   9.]
 [ 13.   4.]]
[[  6.   5.]
 [ 14.  12.]
 [  4.   6.]
 [ 15.  11.]]
[[ 17.   1.]
 [  7.   4.]
 [  8.  15.]
 [ 15.  15.]]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 6, 7)
('column coordinates:', 0, 2)
('maxvol 2x2 submatrix', array([[ 17.,   1.],
       [  8.,  15.]]))
('with det = ', 247.00000000000006)
pivoting and permutations start...
[[ 11.51417004  12.18623482]
 [  3.46558704  -5.5951417 ]]
[[ 11.2388664    9.04048583]
 [  8.56275304   3.04453441]]
[[ 10.6194332   10.51012146]
 [ -1.21862348   1.7611336 ]]
('on the', 2, 'slice')
('best row block is', 2, 3)
('column coordinates:', 2, 3)
('maxvol 2x2 submatrix', array([[ 11.51417004,  12.18623482],
       [  3.46558704,  -5.5951417 ]]))
('with det = ', 106.65587044534415)
pivoting and permutations start...


In [16]:
ids = np.dot(p,np.dot(rara,q))[:4]

In [17]:
A_ids = np.dot(p,np.dot(rara,q))

In [20]:
def perm_array(A):
    p_a = np.array((A.shape[0]))
    p_a = np.argmax(A, axis = 1)
    return p_a


In [21]:
perm_array(p)

array([6, 7, 2, 3, 4, 5, 0, 1])

In [23]:
temp = np.dot(A_ids,np.linalg.inv(ids))

In [24]:
temp

array([[  1.00000000e+00,   0.00000000e+00,  -2.22044605e-16,
          0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,  -1.38777878e-16,
          1.38777878e-16],
       [ -1.11022302e-16,  -1.11022302e-16,   1.00000000e+00,
          0.00000000e+00],
       [ -1.11022302e-16,   0.00000000e+00,   1.11022302e-16,
          1.00000000e+00],
       [ -6.19951412e-01,   4.10188278e-01,   1.56794716e+00,
         -5.59216520e-01],
       [ -2.50227756e-01,   2.80367446e-01,   8.22122684e-01,
         -3.49225630e-02],
       [ -1.37109019e-01,  -2.18797449e-01,   4.17856058e-01,
          4.76617067e-01],
       [ -1.80534467e-01,   4.87928940e-01,   7.52581233e-01,
          1.51381719e-01]])

In [29]:

tol = 0.05
n = temp.shape[0]
m = temp.shape[1]
for k in range(m,n,2):
        pair = np.concatenate((temp[k,:],temp[k+1,:])).reshape(2,m).T
        piv,_ = maxvol(pair)
        piv = np.sort(piv)
        print pair, pair[piv]
        if (np.abs(np.linalg.det(pair[piv])) > (1)):
            print('exist!')

[[-0.61995141 -0.25022776]
 [ 0.41018828  0.28036745]
 [ 1.56794716  0.82212268]
 [-0.55921652 -0.03492256]] [[ 1.56794716  0.82212268]
 [-0.55921652 -0.03492256]]
[[-0.13710902 -0.18053447]
 [-0.21879745  0.48792894]
 [ 0.41785606  0.75258123]
 [ 0.47661707  0.15138172]] [[-0.21879745  0.48792894]
 [ 0.41785606  0.75258123]]
